In [1]:
import pandas as pd, numpy as np, pyDbs
from pyDbs import SymMaps as SM
from pyDbs import adj
from scipy import stats
rng = np.random.default_rng(seed = 105)
stats.truncnorm.random_state = rng

# SymMaps

The class is used to navigate problems that take a vector of inputs $x$ and the math behind it that deals with many different variables potentially defined over different indices. 

## Example

As an example, we'll use the optimization problem defined by:
$$\begin{align}
    \max &\sum_{i, t_0\leq t\leq T} \beta_i^{t-t_0} \omega_i \ln\left(c_{i,t}\right) \tag{1} \\
    m_{i,t} &= R_t m_{i,t-1}-c_{i,t} + y_{i,t}, && \forall  i, t_0< t\leq T \tag{2}\\
    m_{i,T} &= m_{i,T-1}, && \forall i, \tag{3} \\ 
\end{align}$$
given $m_{i,0}, y_{i,t}, R_t, \beta_i, \omega_i$.

As an example, let us consider the simple case with three types of agents $i \in \lbrace L, M, H\rbrace $ and 5 time periods $t = 2010, 2011, 2012,..., 2014$, and let us collect all relevant objects in a database:

In [2]:
db = pyDbs.SimpleDB() # datatbase
# Sets:
db['i'] = pd.Index(['L','M','H'], name  = 'i')
t0, T = 2010, 2014
db['t'] = pd.Index(range(t0, T+1), name = 't')
# Parameters:
db['β'] = pd.Series(sorted(stats.truncnorm.rvs(0, 1, size = len(db['i']))), index = db['i'])
db['ω'] = pd.Series(1/len(db['i']), index = db['i'])
db['y'] = pd.Series(1, index = pd.MultiIndex.from_product([db['i'], db['t']]))
db['R'] = pd.Series(1/db['β'].mean()-0.5+stats.truncnorm.rvs(0,1, size = len(db['t'])), index = db['t'])
db['m0']= pd.Series(10*db['β']**2, index = db['i'])

### Define core symbols

The model is solved by setting up a single vector $x$ that is the stacked representation of vectors **$c, m$** that are defined for all $t,i$. We initialize the stackedPandas object with these two symbols (either as ```pd.Series``` or ```pd.Index```). The ```self.compile``` method defines an index used to navigate the full vector $x$ with a global linear index:

In [3]:
s = SM(symbols = {'c': pd.MultiIndex.from_product([db['i'], db['t']]), 
                   'm': pd.MultiIndex.from_product([db['i'], db['t']])}
       )
s.compile()

As a reference, let us define a stacked vector $x$ as a numpy array that we can slice from to test various method:

In [4]:
x = rng.random(size = 2*len(db['i'])*len(db['t']))

The ```self.compile``` establishes a dictionary of mappings (stored at ```self.maps```) that, for each symbol ($c,m$ in our instance) contains a mapping from the relevant ```pd.Index``` to the numerical indices in the stacked $x$. For the $m$ variable, for instance, this mapping looks as follows:

In [5]:
s['m']

i  t   
L  2010    15
   2011    16
   2012    17
   2013    18
   2014    19
M  2010    20
   2011    21
   2012    22
   2013    23
   2014    24
H  2010    25
   2011    26
   2012    27
   2013    28
   2014    29
dtype: int64

Now, we can use the custom ```__call__``` method (with syntax ```self(k)```) to get the relevant part of the vector $x$:

In [6]:
s(x, 'm') # get the values in vector x that we refer to as the symbol 'm'

array([0.380102  , 0.35163855, 0.21946259, 0.72390011, 0.37699435,
       0.37691073, 0.63893963, 0.34841258, 0.67772011, 0.61233288,
       0.50691337, 0.64130159, 0.63316618, 0.33799887, 0.65869185])

If we prefer to return the object as a ```pd.Series```, we can instead use the ```get``` method:

In [7]:
s.get(x, 'm')

i  t   
L  2010    0.380102
   2011    0.351639
   2012    0.219463
   2013    0.723900
   2014    0.376994
M  2010    0.376911
   2011    0.638940
   2012    0.348413
   2013    0.677720
   2014    0.612333
H  2010    0.506913
   2011    0.641302
   2012    0.633166
   2013    0.337999
   2014    0.658692
dtype: float64

Finally, the ```getr``` method performs the "getting" more robustly; we can for instance pass kwargs used to subset the symbol relying on the ```adj.rc_pd``` method. This gets the $m$ vector, but only keeps the "L" and "M" types:

In [8]:
s.getr(x, 'm', c = pd.Index(['L','M'], name = 'i')) # the 'c' keyword will be parsed to the method adj.rc_pd

i  t   
L  2010    0.380102
   2011    0.351639
   2012    0.219463
   2013    0.723900
   2014    0.376994
M  2010    0.376911
   2011    0.638940
   2012    0.348413
   2013    0.677720
   2014    0.612333
dtype: float64

### Adjusted symbols

In some instances, we need some adjusted version of the symbols $m,c$ that $x$ consists of. For instance, in the example in equations (1)-(3), we need to get the lagged version $m_{i,t-1}$.

* Adjusted symbols are defined using mappings from the original main index to a new adjusted one. The ```pd.Index``` to ```pd.Index``` mapping is stored at ```self.auxMapsIdx```.
* We include default methods for creating lagged, rolled, or shifted indices, and to fill in assumptions of missing values, steady state and similar.
* The compilation stage creates fixed maps to the global linear index and commits this to the ```self.auxMaps``` attribute.
* For dynamically compiled objects, we can always use the ```getr``` method directly with relevant mappings or lags/rolls/shifts in the indices.

#### Roll/shift/lag index

The three methods roll/shift/lag has slighly different uses:
1. Lag: Numerical index levels can be lagged, i.e. the operation is transformed directly on the elements; this can result in new index elements. 
2. Roll: As the name suggests, this uses already defined index levels and rolls them. That is, this relies on the ordering of the main symbol. 
3. Shift: Functions as the Roll, but keeps a break between the first and last element in an index. For example, rolling the index $\lbrace 1,2,3\rbrace$ one element yields $\lbrace 3,1,2 \rbrace$, whereas shifting it leaves $\lbrace NaN, 1,2 \rbrace$ or some other value that we use to fill in for ```NaN```.

##### 1. Lag index:

```pd.MultiIndex```

In [9]:
lags = {'t': 1} # lag 't' index with 1 
symbol = 'm'
name = 'm[t-1]'
kwargs = {'c': ('not', pd.Index([t0], name = 't'))} # 'c' is a condition on the index. Here, don't use initial year t0

Add this as an adjusted symbol that we can access with syntax ```'m[t-1]'```. Note that lagging the index, we end up with values of $t$ that is not defined (in our case $t=2009$) - hence the condition not to include $t_0$:

In [10]:
s.addLaggedSym(name, symbol, lags, **kwargs)

We add this symbol by (1) adding a mapping to ```self.auxMapsIdx``` from original to new index, (2) using this mapping on the "full" symbol already stored in ```self.maps```: 

In [14]:
m = s.lagMaps(adj.rc_pd(s[symbol],**kwargs), lags) # step 1: obtain new mapping - this is added to self.auxMapsIdx
newMap = s.applyMapGlobalIdx(s[symbol], m) # step 2: use mapping to map to global linear index - this is added to self.auxMaps

Compare our new lagged symbol to the original one:

We then apply this

In [ ]:
symbol[m.values] # this returns the symbol with index = [i, t+1]
s.applyMapGlobalIdx(symbol, m) 

KeyError: "[('L', 2009), ('M', 2009), ('H', 2009)] not in index"

We can use this map to 

Note that, before this, the tuple $(i,t) = (L,2010)$ referred to the linear index $15$. Lagging the $t$ index by 1 means that when $t=2011$ it maps to the linear index $15$:

In [10]:
s.lagLevels(symbol, lags)

i  t   
L  2011    15
   2012    16
   2013    17
   2014    18
   2015    19
M  2011    20
   2012    21
   2013    22
   2014    23
   2015    24
H  2011    25
   2012    26
   2013    27
   2014    28
   2015    29
dtype: int64

```pd.Index```:

If we apply the method to a symbol with values (instead of the mapping to a linear index), we return the symbol again but with $t+1$ in the index:

In [21]:
lags =  1 # lag index with 1
symbol = db['R'].copy() # the symbol map to be lagged
pd.concat([symbol.rename('R[t]'), s.lagLevels(symbol, lags).rename('R[t-1]')], axis = 1)

,R[t],R[t-1]
t,,
2010,1.769229,NaN
2011,1.667108,1.769229
2012,1.498320,1.667108
2013,2.168236,1.498320
2014,1.801419,2.168236
2015,NaN,1.801419


*2. Roll index:*

```pd.MultiIndex```

In [12]:
rolls = {'t': 1} # roll 't' index with 1
symbol = s['m'].copy() # the symbol map to be lagged

Roll the yearly index with 1 in the order. That is, where we have '2014' before, we now have '2013' etc. For the first element, '2010', we now map to the last element '2014'. When the index is a range index (e.g. 1, 2, 3...), this is the same as referring to $x[t+1]$ *and* adding that once we reach the final element $T+1\mapsto t_0$ (initial level): 

In [13]:
s.rollLevels(symbol, rolls)

i  t   
L  2014    15
   2010    16
   2011    17
   2012    18
   2013    19
M  2014    20
   2010    21
   2011    22
   2012    23
   2013    24
H  2014    25
   2010    26
   2011    27
   2012    28
   2013    29
dtype: int64

```pd.Index```:

In [14]:
rolls =  1 # roll index with 1
symbol = db['R'].copy() # the symbol map to be lagged
s.rollLevels(symbol, rolls)

t
2014    1.769229
2010    1.667108
2011    1.498320
2012    2.168236
2013    1.801419
dtype: float64

*3. Shift index:*

In [15]:
shifts = {'t': -1}
symbol = s['m'].copy()
kwargs = {'useLoc': 'nn'} # nearest-neighbor loc

Shifting the index by 1 is similar to rolling an index, i.e. it returns the element $x[t+1]$. Instead of using the convention that $T+1\mapsto t_0$, however, we can use other conventions like a steady state assumption here. Using the ```kwargs = {'useLoc': 'nn'} ``` option means that we essentially use the convention $T+1\mapsto T$ in this case.

The method ```shiftMaps``` shows the mapping from $(i,t)$ to the shifted index $(i,t+1)$ with the convention that $T+1\mapsto T$:

In [16]:
m = s.shiftMaps(symbol, shifts, **kwargs)
m

i  t   
L  2010    (L, 2011)
   2011    (L, 2012)
   2012    (L, 2013)
   2013    (L, 2014)
   2014    (L, 2014)
M  2010    (M, 2011)
   2011    (M, 2012)
   2012    (M, 2013)
   2013    (M, 2014)
   2014    (M, 2014)
H  2010    (H, 2011)
   2011    (H, 2012)
   2012    (H, 2013)
   2013    (H, 2014)
   2014    (H, 2014)
dtype: object

As we can see, this generally maps from $t\mapsto t+1$, except for $2014 \mapsto 2014$. When we shift levels in the global linear index, we can do this by drawing directly on this mapping:

In [20]:
symbol[m.values] # this returns the symbol with index = [i, t+1]
s.applyMapGlobalIdx(symbol, m) 

i  t   
L  2010    16
   2011    17
   2012    18
   2013    19
   2014    19
M  2010    21
   2011    22
   2012    23
   2013    24
   2014    24
H  2010    26
   2011    27
   2012    28
   2013    29
   2014    29
dtype: int64

We can also do this without having an established mapping yet; this can be relevant in some reporting routines (where speed is not paramount) or if the numerical part of some optimization requires dynamic compilation (e.g. with adaptive sparse grids where indices may change). This is done by calling the ```shiftLevels``` method:

In [18]:
s.shiftLevels(symbol, shifts, **kwargs)

i  t   
L  2010    16
   2011    17
   2012    18
   2013    19
   2014    19
M  2010    21
   2011    22
   2012    23
   2013    24
   2014    24
H  2010    26
   2011    27
   2012    28
   2013    29
   2014    29
dtype: int64

```pd.Index```:

In [24]:
shifts = -1 # shift index with -1
symbol = db['R'].copy() # the symbol map to be lagged
pd.DataFrame({'R[t]': symbol, 'R[t+1]': s.shiftLevels(symbol, shifts, **kwargs)})

,R[t],R[t+1]
t,,
2010,2.462717,2.540789
2011,2.540789,3.050615
2012,3.050615,3.276308
2013,3.276308,2.834768
2014,2.834768,2.834768


#### Compile adjusted symbols

As mentioned above, the adjusted symbols are compiled in two steps: First, by adding a mapping to ```self.auxMapsIdx``` from the original ```pd.Index``` to a new one. Second, by 
adding to ```self.auxMaps``` a mapping from the original ```pd.Index``` to the numerical index required to extract the relevant entries in a stacked numpy array (akin to the ```self.maps``` dict).

*Compiling a lagged, rolled, or shifted variable:*

Let us look for a way to extract $m_{i,t-1}$. We can get this map by lagging the $t$-index by 1:

In [18]:
lags = {'t': 1}
symbol = s['m'].copy()
m = s.lagMaps(symbol, lags)
m

i  t   
L  2010    (L, 2011)
   2011    (L, 2012)
   2012    (L, 2013)
   2013    (L, 2014)
   2014    (L, 2015)
M  2010    (M, 2011)
   2011    (M, 2012)
   2012    (M, 2013)
   2013    (M, 2014)
   2014    (M, 2015)
H  2010    (H, 2011)
   2011    (H, 2012)
   2012    (H, 2013)
   2013    (H, 2014)
   2014    (H, 2015)
dtype: object

This map is used to indicate that whenever the index $(i,t) = (L, 2010)$, this symbol operates over $(L,2009)$ etc.. However, as the value $t=2009$ is not in the original index, we run into a problem if we try to use this to establish a mapping to the global linear index. In particular, because the final lagged value $2009$ does not exist here. So, looking up the 